In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import EarlyStopping

2024-03-27 10:15:51.864614: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-27 10:15:51.965977: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-27 10:15:52.598334: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-27 10:15:55.536615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def RSI(data, time_period=14, column='close', MA='SMA'):
    delta = data[column].diff(1)
    delta = delta.dropna()
    up = delta.copy()
    down = delta.copy()
    up[up < 0 ] = 0
    down[down > 0] = 0
    if MA == 'SMA':
        AVG_gain = up.rolling(window=time_period).mean()
        AVG_loss = abs(down.rolling(window=time_period).mean())
    elif MA == 'EMA':
        AVG_gain = up.ewm(span=time_period, adjust=False).mean()
        AVG_loss = abs(down.ewm(span=time_period, adjust=False).mean())
        
    rs = AVG_gain / AVG_loss
    rsi = 100 - (100 / (1+rs))
    data['RSI'+str(time_period)+'_'+str(MA)] = rsi

In [87]:
class LogisticRegression_tf(tf.keras.Model):
    def __init__(self, num_features):
        super(LogisticRegression, self).__init__()
        self.w = tf.Variable(tf.random.normal(shape=(num_features, 1)), trainable=True)
        self.b = tf.Variable(tf.zeros(shape=(1,)), trainable=True)

    def call(self, inputs):
        return tf.sigmoid(tf.matmul(inputs, self.w) + self.b)

    def compile(self, optimizer, loss):
        super(LogisticRegression, self).compile(optimizer=optimizer, loss=loss)
        self.optimizer = optimizer
        self.loss_fn = loss

    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.loss_fn(y, y_pred)
        gradients = tape.gradient(loss, [self.w, self.b])
        self.optimizer.apply_gradients(zip(gradients, [self.w, self.b]))
        return {"loss": loss}
    
    def score(self, X, y):
        y_pred = self.predict(X)
        y_pred_classes = tf.round(y_pred)
        accuracy = tf.reduce_mean(tf.cast(tf.equal(y, y_pred_classes), tf.float32))
        return accuracy

In [84]:
def select_best_model_tf(X_train, Y_train):
    num_features = X_train.shape[1]
    LG = LogisticRegression_tf(num_features)
    LG.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.BinaryCrossentropy())
    
    model_svc = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model_svc.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
    
    models = [
        model_svc,
        LG
    ]
    
    model_scores = []
    for model in models:
        print('Model')
        model.fit(X_train,Y_train)
        score = model.score(X_train,Y_train)
        model_scores.append((model, score))
        
    # Select best Model
    best_model = max(model_scores, key = lambda x:x[1])[0]
    
    return best_model

In [3]:
def select_best_model_sk(X_train, Y_train):

    models = [
        RandomForestClassifier(),
        SVC(),
        LogisticRegression()
    ]
    
    model_scores = []
    for model in models:
        print('Model')
        model.fit(X_train,Y_train)
        score = model.score(X_train,Y_train)
        print('Model {score}')
        model_scores.append((model, score))
        
    # Select best Model
    best_model = max(model_scores, key = lambda x:x[1])[0]
    
    return best_model

In [4]:
#Load Data
df = pd.read_csv('bnb_historical_data.csv')

df['percent_change'] = df['close'].pct_change()
## Signal on percent change
df['signal'] = np.where(df['percent_change'] >= 0, 1, -1)
df.dropna(inplace=True)

RSI(df, 6, MA='SMA')
RSI(df, 2, MA='SMA')
df.dropna(inplace=True)

## Signal on RSI
df['RSI6_signal'] = np.where(df['RSI6_SMA'] <= 10, 1, 0)
df['RSI6_signal'] = np.where(df['RSI6_SMA'] >= 90, -1, df['RSI6_signal'])

df['RSI2_signal'] = np.where(df['RSI2_SMA'] <= 10, 1, 0)
df['RSI2_signal'] = np.where(df['RSI2_SMA'] >= 90, -1, df['RSI2_signal'])

df.set_index(df['timestamp'])
df.drop('timestamp', axis=1, inplace=True)

In [5]:
features = ['volume', 'close', 'RSI6_signal', 'RSI2_signal']
target = ['signal']

X = df[features]
Y = df[target]

# Split data for training and testing (80:20)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=0)

best_model = select_best_model_sk(X_train, Y_train)

y_pred = best_model.predict(X_test)
score = best_model.score(X_test, Y_test)

print(f"Accuracy of the best model: {score}")

Model


/home/jack/codes/ML_testing/.venv/lib/python3.10/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Model


/home/jack/codes/ML_testing/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model


/home/jack/codes/ML_testing/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of the best model: 0.7122029245517525
